https://qiita.com/Ken-Kuroki/items/115d0679fdce1d15b9d1

https://github.com/bicycle1885/Julia-Tutorial

In [36]:
Pkg.add("Query")

   Updating registry at `/opt/julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %==========>                             ]  25.1 %===============>                         ]  36.9 %Fetching: [================>                        ]  39.0 %>              ]  63.9 %Fetching: [=======================================> ]  95.2 %

  Resolving package versions...
  Installed TableShowUtils ─── v0.2.5
  Installed TableTraitsUtils ─ v1.0.1
  Installed IterableTables ─── v1.0.0
  Installed QueryOperators ─── v0.9.2
  Installed Query ──────────── v1.0.0
  Installed DataValues ─────── v0.4.13
   Updating `/opt/julia/environments/v1.4/Project.toml`
  [1a8c2f83] + Query v1.0.0
   Updating `/opt/julia/environments/v1.4/Manifest.toml`
  [e7dc6d0d] + DataValues v0.4.13
  [1c8ee90f] + IterableTables v1.0.0
  [1a8c2f83] + Query v1.0.0
  [2aef5ad7] + QueryOperators v0.9.2
  [5e66a065] + TableShowUtils v0.2.5
  [382cd787] + TableTraitsUtils v1.0.1


In [1]:
ENV["COLUMNS"]=240  # 描画する表の列数を増やす
ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）

using Pkg

#Pkg.add("DataFrames")
#Pkg.add("DataFramesMeta")
#Pkg.add("LibPQ")
#Pkg.add("Query")
#Pkg.add("StatsBase")
#Pkg.add("ScikitLearn")
#Pkg.add("MLDataPattern")
#Pkg.add("CSV")

using DataFrames
using DataFramesMeta
using LibPQ
using Query
using StatsBase
using Statistics
using Dates
using Random
using ScikitLearn
using MLDataPattern
using CSV

@sk_import preprocessing: (LabelBinarizer, StandardScaler, MinMaxScaler)

PyObject <class 'sklearn.preprocessing._data.MinMaxScaler'>

# データサイエンス100本ノック（構造化データ加工編）

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [5]:

fill(42, 10)

10-element Array{Int64,1}:
 42
 42
 42
  ⋮
 42
 42

In [13]:
# list
x = [1, 2, 3]
push!(x, 4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [14]:
issorted([1,2,3,4])

true

In [15]:
filter(x->x>2, x)

2-element Array{Int64,1}:
 3
 4

In [16]:
# dict
d = Dict("one" => 1, "two" => 2)
haskey(d, "two")

true

In [17]:
d["one"]

1

In [18]:
get(d, "three", 0)

0

In [76]:
a = [1,2]
a[1]

1

In [69]:
# set
s = Set([1,2,3,1])
2 in s

true

In [21]:
# 関数の適用
map(sin, [0.0, 1.0, 2.0, 3.0])

4-element Array{Float64,1}:
 0.0
 0.8414709848078965
 0.9092974268256817
 0.1411200080598672

# 演習問題

### load data

In [23]:
pwd()

"/home/jovyan/julia/100knocks-preprocess-master/docker/work"

In [2]:
df_category = CSV.read("./data/category.csv")
df_customer = CSV.read("./data/customer.csv")
df_geocode = CSV.read("./data/geocode.csv")
df_product = CSV.read("./data/product.csv")
df_receipt = CSV.read("./data/receipt.csv")
df_store = CSV.read("./data/store.csv")


┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `using DataFrames; CSV.read(input, DataFrame; kw...)
│   caller = read(::String) at CSV.jl:40
└ @ CSV /opt/julia/packages/CSV/MKemC/src/CSV.jl:40


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String,Int64,String,String,String,String,Float64,Float64,Float64
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.6356,1698.0
2,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.48,35.7057,1735.0
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
6,S13051,板橋大原店,13,東京都,東京都板橋区大原町,トウキョウトイタバシクオオハラチョウ,03-0123-4029,139.698,35.7679,1045.0
7,S13015,南砂店,13,東京都,東京都江東区南砂二丁目,トウキョウトコウトウクミナミスナニチョウメ,03-0123-4014,139.823,35.6707,1337.0
8,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
9,S13044,南六郷店,13,東京都,東京都大田区南六郷二丁目,トウキョウトオオタクミナミロクゴウニチョウメ,03-0123-4028,139.721,35.546,1379.0


---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [37]:
first(df_receipt, 10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64
1,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [44]:
first(df_receipt[:,["sales_ymd", "customer_id", "product_cd", "amount"]], 10)


,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [47]:
first(rename(df_receipt,"sales_ymd" => "sales_date")[:,["sales_date", "customer_id", "product_cd", "amount"]], 10)


,sales_date,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [70]:
# 行の条件、列の条件
df_receipt[df_receipt."customer_id".=="CS018205000001",
    ["sales_ymd", "customer_id", "product_cd", "amount"]]

,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [75]:
# 行の条件、列の条件
df_receipt[(df_receipt."customer_id" .== "CS018205000001") .& (df_receipt."amount" .>= 1000),
    ["sales_ymd", "customer_id", "product_cd", "amount"]]


,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401012,2200
2,20190226,CS018205000001,P071401020,2200
3,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [85]:
# 行の条件、列の条件
df_receipt[(df_receipt."customer_id" .== "CS018205000001") .&
    ((df_receipt."amount" .>= 1000) .| (df_receipt."quantity" .>= 5)),
    ["sales_ymd", "customer_id", "product_cd", "quantity", "amount"]]


,sales_ymd,customer_id,product_cd,quantity,amount
,Int64,String,String,Int64,Int64
1,20180911,CS018205000001,P071401012,1,2200
2,20180414,CS018205000001,P060104007,6,600
3,20170614,CS018205000001,P050206001,5,990
4,20190226,CS018205000001,P071401020,1,2200
5,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [89]:
# 行の条件、列の条件
df_receipt[(df_receipt."customer_id" .== "CS018205000001") .&
    (df_receipt."amount" .>= 1000) .& (df_receipt."amount" .<= 2000),
    ["sales_ymd", "customer_id", "product_cd", "amount"]]


,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64
1,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [96]:
# 行の条件、列の条件
df_receipt[(df_receipt."customer_id" .== "CS018205000001") .&
    (df_receipt."product_cd" .!= "P071401019"),
    ["sales_ymd", "customer_id", "product_cd", "amount"]]

,sales_ymd,customer_id,product_cd,amount
,Int64,String,String,Int64


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [104]:
# notA or B = A and notB
df_store[(df_store."prefecture_cd" .!= 13) .&
    (df_store."floor_area" .<= 900),:]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String,Int64,String,String,String,String,Float64,Float64,Float64
1,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
2,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
3,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [113]:
# ismatch的な関数
first(df_store[[occursin(r"S14", s) for s in df_store[:,"store_cd"]], :], 10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String,Int64,String,String,String,String,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.579,35.2964,1838.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [116]:
# ismatch的な関数
first(df_customer[[occursin(r"1$", s) for s in df_customer[:,"customer_id"]], :], 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [119]:
df_store[[occursin(r".*横浜市.*", s) for s in df_store[:,"address"]], :]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String,Int64,String,String,String,String,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
4,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
5,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
6,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
7,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
8,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
9,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.576,35.5491,1657.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [121]:
first(df_customer[[occursin(r"^[A-F]", s) for s in df_customer[:,"status_cd"]], :], 10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [125]:
first(df_customer[[occursin(r"[1-9]$", s) for s in df_customer[:,"status_cd"]], :], 10)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [126]:
first(df_customer[[occursin(r"^[A-F].*[1-9]$", s) for s in df_customer[:,"status_cd"]], :], 10)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [132]:
df_store[[occursin(r"[0-9]{3}-[0-9]{3}-[0-9]{4}$", s) for s in df_store[:,"tel_no"]], :]


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String,String,Int64,String,String,String,String,Float64,Float64,Float64
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.6356,1698.0
2,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.48,35.7057,1735.0
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
6,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
7,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
8,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.438,35.5529,1087.0
9,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [8]:
first(sort(df_customer, "birth_day"),10)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [10]:
first(sort(df_customer, "birth_day", rev=true), 10)


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String,String,Int64,String,Date,Int64,String,String,String,Int64,String
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [17]:
#sort(ordinalrank(df_receipt[:,"amount"], rev=false))
sort()


104681-element Array{Int64,1}:
      1
      2
      3
      ⋮
 104512
 104512

In [26]:
first(
    sort(rename(hcat(df_receipt, competerank(df_receipt[:,"amount"], rev=true)),"x1"=>"rank"), "amount", rev=true)
, 10)


,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,rank
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64
1,20190903,1283472000,S14011,1192,2,CS011415000006,P080104055,1,10925,1
2,20190304,1267660800,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2
3,20170921,1221955200,S14028,1192,2,CS028605000002,P090907210,1,5780,3
4,20170721,1216598400,S13015,1112,1,CS015515000034,P090907208,1,5480,4
5,20190103,1262476800,S14021,1162,2,ZZ000000000000,P090907208,1,5480,4
6,20190724,1279929600,S13009,1192,2,ZZ000000000000,P090907209,1,5480,4
7,20180621,1245542400,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7
8,20190209,1265673600,S14021,1142,2,CS021515000089,P090907206,1,5440,7
9,20181219,1261180800,S13015,1132,1,CS015515000083,P071301009,1,5280,9


In [55]:
# 別解 関数の入れ子が辛くなってきたので
@linq df_receipt |>
    sort(order(:amount, rev=true)) |>
    transform(rank=competerank(:amount, rev=true)) |>
    select(:customer_id, :amount, :rank) |>
    first(10)

,customer_id,amount,rank
,String,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,7
8,CS021515000089,5440,7
9,CS015515000083,5280,9


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [61]:
@linq df_receipt |>
    sort(order(:amount, rev=true)) |>
    transform(rank=ordinalrank(:amount, rev=true)) |>
    select(:customer_id, :amount, :rank) |>
    first(10)

,customer_id,amount,rank
,String,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [97]:
length(df_receipt.customer_id)

104681

In [98]:
nrow(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [96]:
length(unique(df_receipt.customer_id))

8307

In [ ]:
nrow(df_receipt)

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [104]:
@linq df_receipt |>
    select(:store_cd, :amount, :quantity) |>
    groupby(:store_cd) |>
    combine(:amount => sum, :quantity => sum) |>
    orderby(:store_cd)

,store_cd,amount_sum,quantity_sum
,String,Int64,Int64
1,S12007,638761,2099
2,S12013,787513,2425
3,S12014,725167,2358
4,S12029,794741,2555
5,S12030,684402,2403
6,S13001,811936,2347
7,S13002,727821,2340
8,S13003,764294,2197
9,S13004,779373,2390


In [107]:
@linq df_receipt |>
  groupby(:store_cd) |>
  transform(sum_amount=sum(:amount), sum_quantity=sum(:quantity)) |>
  unique(:store_cd) |>
  orderby(:store_cd) |>
  select(:store_cd, :sum_amount, :sum_quantity)

,store_cd,sum_amount,sum_quantity
,String,Int64,Int64
1,S12007,638761,2099
2,S12013,787513,2425
3,S12014,725167,2358
4,S12029,794741,2555
5,S12030,684402,2403
6,S13001,811936,2347
7,S13002,727821,2340
8,S13003,764294,2197
9,S13004,779373,2390


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [120]:
@linq df_receipt |>
  select(:customer_id, :sales_ymd) |>
  orderby(:customer_id, :sales_ymd) |>
  groupby(:customer_id) |>
  combine(:sales_ymd => last) |>
  first(10)

,customer_id,sales_ymd_last
,String,Int64
1,CS001113000004,20190308
2,CS001114000005,20190731
3,CS001115000010,20190405
4,CS001205000004,20190625
5,CS001205000006,20190224
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [123]:
@linq df_receipt |>
  select(:customer_id, :sales_ymd) |>
  orderby(:customer_id, :sales_ymd) |>
  groupby(:customer_id) |>
  combine(:sales_ymd => first) |>
  first(10)

,customer_id,sales_ymd_first
,String,Int64
1,CS001113000004,20190308
2,CS001114000005,20180503
3,CS001115000010,20171228
4,CS001205000004,20170914
5,CS001205000006,20180207
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

# これで１００本終わりです。おつかれさまでした！